In [15]:
import numpy as np
import pandas as pd
import csv
import re
from Bio import SeqIO
import os

# create GO-list tab delimeted  from .obo format

In [75]:
file = open("GOlist.obo", "r")

In [76]:
GO_df = pd.DataFrame()
_id = ""
name = ""
namespace = ""
_def = ""
synonym = []
_term = False

In [77]:
for line in file:
    if line == "\n":
        if _term == False:    
            pass
        elif _term == True:
            # save the previous one
            GO_df = GO_df.append(pd.DataFrame({'ID': _id,'name':name , "namespace":namespace ,"definition" : _def}, index=[0]), ignore_index=True)
            # start the new record
            _id = ""
            name= ""
            namespace = ""
            _def = ""
            _term = False
        continue
        
    line = line.replace('\n', '')
    
    if line.startswith('[Term]'):
        _term = True
              
    if line.startswith('id:'):
        x = line.split()
        _id = x[1] # get id 
        
    if line.startswith('name:'):
        x = line.split(" ", 1)
        name = x[1] # get name
    
    if line.startswith('namespace:'):
        x = line.split(" ", 1)
        namespace = x[1] # get namespace
    
    if line.startswith('def:'):
        x = re.findall(r'"([^"]*)"', line)
        _def = x[0] # get def
    
    #Keep the last one
   # if line.startswith('[Typedef]'):
   #     GO_df = GO_df.append(pd.DataFrame({'ID': _id,'name':name , "name_sapce":namespace ,"definition" : _def}, index=[0]), ignore_index=True)
        

In [78]:
# GO_df.loc[4]["definition"]
GO_df

,ID,name,namespace,definition
0,GO:0000001,mitochondrion inheritance,biological_process,"The distribution of mitochondria, including th..."
1,GO:0000002,mitochondrial genome maintenance,biological_process,The maintenance of the structure and integrity...
2,GO:0000003,reproduction,biological_process,The production of new individuals that contain...
3,GO:0000005,obsolete ribosomal chaperone activity,molecular_function,OBSOLETE. Assists in the correct assembly of r...
4,GO:0000006,high-affinity zinc transmembrane transporter a...,molecular_function,Enables the transfer of zinc ions (Zn2+) from ...
...,...,...,...,...
47205,GO:2001313,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,The chemical reactions and pathways involving ...
47206,GO:2001314,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,The chemical reactions and pathways resulting ...
47207,GO:2001315,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,The chemical reactions and pathways resulting ...
47208,GO:2001316,kojic acid metabolic process,biological_process,The chemical reactions and pathways involving ...


In [81]:
GO_df.to_csv("GOTerms.csv", columns=GO_df.columns ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)

## Create AMP GO term

In [138]:
## Find GO Id that contains antibacterial,antimicrobial, antifungal, antiviral, defense, immun,  anticancer
_tmp = GO_df[GO_df['definition'].str.contains("antibacterial|antimicrobial|antifungal|antiviral|defense|defence|immune|immunity|immunological|anticancer")]
_tmp

,ID,name,namespace,definition
1085,GO:0001776,leukocyte homeostasis,biological_process,The process of regulating the proliferation an...
1092,GO:0001783,B cell apoptotic process,biological_process,"Any apoptotic process in a B cell, a lymphocyt..."
1110,GO:0001802,type III hypersensitivity,biological_process,An inflammatory response resulting from recogn...
1128,GO:0001820,serotonin secretion,biological_process,The regulated release of serotonin by a cell. ...
1154,GO:0001846,opsonin binding,molecular_function,Interacting selectively and non-covalently wit...
...,...,...,...,...
46440,GO:2000522,positive regulation of immunological synapse f...,biological_process,Any process that activates or increases the fr...
46444,GO:2000526,positive regulation of glycoprotein biosynthet...,biological_process,Any positive regulation of glycoprotein biosyn...
47086,GO:2001191,regulation of gamma-delta T cell activation in...,biological_process,"Any process that modulates the frequency, rate..."
47087,GO:2001192,negative regulation of gamma-delta T cell acti...,biological_process,"Any process that stops, prevents or reduces th..."


In [89]:
# _tmp = GO_df[GO_df['definition'].str.contains("anti|immun|defense")] <-- incase create  super terms

In [139]:
_tmp.to_csv("amp_tmp.csv", columns=GO_df.columns ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)

# Check with groud truth

In [5]:
amp_go_df = pd.read_csv('GoTerm/amp_GO.csv', sep=",", header=0,quoting=csv.QUOTE_ALL)  
amp_go_stringent_df = pd.read_csv('GoTerm/amp_stringent_GO.csv', sep=",", header=0,quoting=csv.QUOTE_ALL)  

## Bat

In [8]:
anno_BAT = pd.read_csv('truth/BAT_annotation_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_BAT['all_GO'] = anno_BAT['gene_ontology_BLASTX'] + '`' + anno_BAT['gene_ontology_BLASTP'] + "`" +  anno_BAT['gene_ontology_Pfam']
BAT_truth_set = anno_BAT[ anno_BAT.all_GO.str.contains('|'.join(amp_go_df.ID.to_list()))]
BAT_truth_set = BAT_truth_set[BAT_truth_set["prot_id"] != "."]
BAT_truth_set.to_csv("truth/BAT_truth_set.csv", columns=BAT_truth_set.columns ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)
BAT_truth_set

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,uniprot_metazoa_33208.fasta_BLASTX,uniprot_metazoa_33208.fasta_BLASTP,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_BLASTX,gene_ontology_BLASTP,gene_ontology_Pfam,transcript,peptide,all_GO
7,TRINITY_DN18674_c0_g1_i1,TRINITY_DN18674_c0_g1_i1,"TRAD1_BOVIN^TRAD1_BOVIN^Q:124-738,H:1-204^87.3...",.,TRINITY_DN18674_c0_g1_i1.p1,124-741[+],"TRAD1_BOVIN^TRAD1_BOVIN^Q:1-205,H:1-204^87.3%I...",tr|L5LJI1|L5LJI1_MYODS^tr|L5LJI1|L5LJI1_MYODS^...,tr|L5LJI1|L5LJI1_MYODS^tr|L5LJI1|L5LJI1_MYODS^...,.,.,.,.,KEGG:bta:512642,GO:0005739^cellular_component^mitochondrion`GO...,GO:0005739^cellular_component^mitochondrion`GO...,.,.,.,GO:0005739^cellular_component^mitochondrion`GO...
9,TRINITY_DN18675_c0_g1_i1,TRINITY_DN18675_c0_g1_i1,"IRF1_BOVIN^IRF1_BOVIN^Q:399-214,H:1-62^98.4%ID...",.,TRINITY_DN18675_c0_g1_i1.p1,1-399[-],"IRF1_BOVIN^IRF1_BOVIN^Q:1-62,H:1-62^98.4%ID^E:...",tr|A0A673U5G9|A0A673U5G9_SURSU^tr|A0A673U5G9|A...,tr|A0A384C647|A0A384C647_URSMA^tr|A0A384C647|A...,PF00605.18^IRF^Interferon regulatory factor tr...,.,.,.,KEGG:bta:789216,GO:0005737^cellular_component^cytoplasm`GO:000...,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.,GO:0005737^cellular_component^cytoplasm`GO:000...
34,TRINITY_DN4863_c0_g1_i1,TRINITY_DN4863_c0_g1_i1,"NECT1_PIG^NECT1_PIG^Q:265-1074,H:15-284^85.9%I...",.,TRINITY_DN4863_c0_g1_i1.p4,1088-1750[+],"NECT1_HUMAN^NECT1_HUMAN^Q:1-221,H:32-252^87.3%...",tr|G1NT63|G1NT63_MYOLU^tr|G1NT63|G1NT63_MYOLU^...,tr|L5M6Q8|L5M6Q8_MYODS^tr|L5M6Q8|L5M6Q8_MYODS^...,PF00047.26^ig^Immunoglobulin domain^5-99^E:1.1...,.,.,.,KEGG:hsa:5818,GO:0005912^cellular_component^adherens junctio...,GO:0005912^cellular_component^adherens junctio...,.,.,.,GO:0005912^cellular_component^adherens junctio...
91,TRINITY_DN5330_c0_g1_i1,TRINITY_DN5330_c0_g1_i1,"HCK_MOUSE^HCK_MOUSE^Q:712-290,H:1-141^73.8%ID^...",.,TRINITY_DN5330_c0_g1_i1.p4,474-809[+],.,tr|F1PSI2|F1PSI2_CANLF^tr|F1PSI2|F1PSI2_CANLF^...,.,.,.,.,.,KEGG:mmu:15162,GO:0005901^cellular_component^caveola`GO:00429...,.,.,.,.,GO:0005901^cellular_component^caveola`GO:00429...
92,TRINITY_DN5330_c0_g1_i1,TRINITY_DN5330_c0_g1_i1,"HCK_MOUSE^HCK_MOUSE^Q:712-290,H:1-141^73.8%ID^...",.,TRINITY_DN5330_c0_g1_i1.p1,1-459[-],"HCK_MACFA^HCK_MACFA^Q:58-153,H:157-252^90.6%ID...",tr|F1PSI2|F1PSI2_CANLF^tr|F1PSI2|F1PSI2_CANLF^...,tr|L5LNS5|L5LNS5_MYODS^tr|L5LNS5|L5LNS5_MYODS^...,PF00017.25^SH2^SH2 domain^58-105^E:3.5e-10,.,.,.,KEGG:mcf:102115729,GO:0005901^cellular_component^caveola`GO:00429...,GO:0005901^cellular_component^caveola`GO:00058...,.,.,.,GO:0005901^cellular_component^caveola`GO:00429...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21174,SOAP.k61.scaffold787,SOAP.k61.scaffold787,"TRI13_HUMAN^TRI13_HUMAN^Q:1292-75,H:1-407^91.6...",.,SOAP.k61.scaffold787.p2,1360-1686[-],.,tr|G1Q7V7|G1Q7V7_MYOLU^tr|G1Q7V7|G1Q7V7_MYOLU^...,.,.,.,.,.,KEGG:hsa:10206,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.,.,GO:0005737^cellular_component^cytoplasm`GO:000...
21175,SOAP.k61.scaffold787,SOAP.k61.scaffold787,"TRI13_HUMAN^TRI13_HUMAN^Q:1292-75,H:1-407^91.6...",.,SOAP.k61.scaffold787.p1,72-1292[-],"TRI13_HUMAN^TRI13_HUMAN^Q:1-406,H:1-407^91.6%I...",tr|G1Q7V7|G1Q7V7_MYOLU^tr|G1Q7V7|G1Q7V7_MYOLU^...,tr|G1Q7V7|G1Q7V7_MYOLU^tr|G1Q7V7|G1Q7V7_MYOLU^...,"PF13920.7^zf-C3HC4_3^Zinc finger, C3HC4 type (...",.,.,.,KEGG:hsa:10206,GO:0005737^cellular_component^cytoplasm`GO:000...,GO:0005737^cellular_component^cytoplasm`GO:000...,GO:0046872^molecular_function^metal ion bindin...,.,.,GO:0005737^cellular_component^cytoplasm`GO:000...
21296,SOAP.k61.C37063,SOAP.k61.C37063,"FAK1_HUMAN^FAK1_HUMAN^Q:339-1,H:678-790^89.4%I...",.,SOAP.k61.C37063.p2,206-520[-],"FAK1_CHICK^FAK1_CHICK^Q:1-60,H:774-833^81.7%ID...",tr|L5MIB4|L5MIB4_MYODS^tr|L5MIB4|L5MIB4_MYODS^...,tr|A0A3Q0DK70|A0A3Q0DK70_CARSF^tr|A0A3Q0DK70|A...,.,.,.,.,KEGG:gga:396416,GO:0005938^cellular_component^cell cortex`GO:0...,GO:0036064^cellular_componen

In [12]:
# # amp_go_stringent_df
anno_BAT = pd.read_csv('truth/BAT_annotation_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_BAT['all_GO'] = anno_BAT['gene_ontology_BLASTX'] + '`' + anno_BAT['gene_ontology_BLASTP'] + "`" +  anno_BAT['gene_ontology_Pfam']
BAT_truth_set = anno_BAT[ anno_BAT.all_GO.str.contains('|'.join(amp_go_stringent_df.ID.to_list()))]
BAT_truth_set = BAT_truth_set[BAT_truth_set["prot_id"] != "."]
BAT_truth_set.to_csv("truth/BAT_truth_set.stringent.csv", columns=BAT_truth_set.columns ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)
BAT_truth_set

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,uniprot_metazoa_33208.fasta_BLASTX,uniprot_metazoa_33208.fasta_BLASTP,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_BLASTX,gene_ontology_BLASTP,gene_ontology_Pfam,transcript,peptide,all_GO
91,TRINITY_DN5330_c0_g1_i1,TRINITY_DN5330_c0_g1_i1,"HCK_MOUSE^HCK_MOUSE^Q:712-290,H:1-141^73.8%ID^...",.,TRINITY_DN5330_c0_g1_i1.p4,474-809[+],.,tr|F1PSI2|F1PSI2_CANLF^tr|F1PSI2|F1PSI2_CANLF^...,.,.,.,.,.,KEGG:mmu:15162,GO:0005901^cellular_component^caveola`GO:00429...,.,.,.,.,GO:0005901^cellular_component^caveola`GO:00429...
92,TRINITY_DN5330_c0_g1_i1,TRINITY_DN5330_c0_g1_i1,"HCK_MOUSE^HCK_MOUSE^Q:712-290,H:1-141^73.8%ID^...",.,TRINITY_DN5330_c0_g1_i1.p1,1-459[-],"HCK_MACFA^HCK_MACFA^Q:58-153,H:157-252^90.6%ID...",tr|F1PSI2|F1PSI2_CANLF^tr|F1PSI2|F1PSI2_CANLF^...,tr|L5LNS5|L5LNS5_MYODS^tr|L5LNS5|L5LNS5_MYODS^...,PF00017.25^SH2^SH2 domain^58-105^E:3.5e-10,.,.,.,KEGG:mcf:102115729,GO:0005901^cellular_component^caveola`GO:00429...,GO:0005901^cellular_component^caveola`GO:00058...,.,.,.,GO:0005901^cellular_component^caveola`GO:00429...
135,TRINITY_DN18336_c0_g1_i2,TRINITY_DN18336_c0_g1_i2,"RN166_HUMAN^RN166_HUMAN^Q:1198-545,H:20-237^95...",.,TRINITY_DN18336_c0_g1_i2.p1,542-1198[-],"RN166_HUMAN^RN166_HUMAN^Q:1-218,H:20-237^95.9%...",tr|F6Q5Y4|F6Q5Y4_HORSE^tr|F6Q5Y4|F6Q5Y4_HORSE^...,tr|F6Q5Y4|F6Q5Y4_HORSE^tr|F6Q5Y4|F6Q5Y4_HORSE^...,"PF13920.7^zf-C3HC4_3^Zinc finger, C3HC4 type (...",.,.,.,KEGG:hsa:115992,GO:0005737^cellular_component^cytoplasm`GO:004...,GO:0005737^cellular_component^cytoplasm`GO:004...,GO:0046872^molecular_function^metal ion binding,.,.,GO:0005737^cellular_component^cytoplasm`GO:004...
136,TRINITY_DN18336_c0_g1_i2,TRINITY_DN18336_c0_g1_i2,"RN166_HUMAN^RN166_HUMAN^Q:1198-545,H:20-237^95...",.,TRINITY_DN18336_c0_g1_i2.p3,134-490[-],.,tr|F6Q5Y4|F6Q5Y4_HORSE^tr|F6Q5Y4|F6Q5Y4_HORSE^...,.,.,.,.,.,KEGG:hsa:115992,GO:0005737^cellular_component^cytoplasm`GO:004...,.,.,.,.,GO:0005737^cellular_component^cytoplasm`GO:004...
145,TRINITY_DN24680_c0_g1_i1,TRINITY_DN24680_c0_g1_i1,"MSRB1_HUMAN^MSRB1_HUMAN^Q:2-298,H:16-114^93.9%...",.,TRINITY_DN24680_c0_g1_i1.p1,37-357[-],.,tr|A0A1S3A245|A0A1S3A245_ERIEU^tr|A0A1S3A245|A...,.,.,.,.,.,KEGG:hsa:51734,GO:0005856^cellular_component^cytoskeleton`GO:...,.,.,.,.,GO:0005856^cellular_component^cytoskeleton`GO:...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21174,SOAP.k61.scaffold787,SOAP.k61.scaffold787,"TRI13_HUMAN^TRI13_HUMAN^Q:1292-75,H:1-407^91.6...",.,SOAP.k61.scaffold787.p2,1360-1686[-],.,tr|G1Q7V7|G1Q7V7_MYOLU^tr|G1Q7V7|G1Q7V7_MYOLU^...,.,.,.,.,.,KEGG:hsa:10206,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.,.,GO:0005737^cellular_component^cytoplasm`GO:000...
21175,SOAP.k61.scaffold787,SOAP.k61.scaffold787,"TRI13_HUMAN^TRI13_HUMAN^Q:1292-75,H:1-407^91.6...",.,SOAP.k61.scaffold787.p1,72-1292[-],"TRI13_HUMAN^TRI13_HUMAN^Q:1-406,H:1-407^91.6%I...",tr|G1Q7V7|G1Q7V7_MYOLU^tr|G1Q7V7|G1Q7V7_MYOLU^...,tr|G1Q7V7|G1Q7V7_MYOLU^tr|G1Q7V7|G1Q7V7_MYOLU^...,"PF13920.7^zf-C3HC4_3^Zinc finger, C3HC4 type (...",.,.,.,KEGG:hsa:10206,GO:0005737^cellular_component^cytoplasm`GO:000...,GO:0005737^cellular_component^cytoplasm`GO:000...,GO:0046872^molecular_function^metal ion bindin...,.,.,GO:0005737^cellular_component^cytoplasm`GO:000...
21296,SOAP.k61.C37063,SOAP.k61.C37063,"FAK1_HUMAN^FAK1_HUMAN^Q:339-1,H:678-790^89.4%I...",.,SOAP.k61.C37063.p2,206-520[-],"FAK1_CHICK^FAK1_CHICK^Q:1-60,H:774-833^81.7%ID...",tr|L5MIB4|L5MIB4_MYODS^tr|L5MIB4|L5MIB4_MYODS^...,tr|A0A3Q0DK70|A0A3Q0DK70_CARSF^tr|A0A3Q0DK70|A...,.,.,.,.,KEGG:gga:396416,GO:0005938^cellular_component^cell cortex`GO:0...,GO:0036064^cellular_component^ciliary basal bo...,.,.,.,GO:0005938^cellular_component^cell cortex`GO:0...
21306,SOAP.k61.C38569,SOAP.k61.C38569,"S10A9_BOVIN^S10A9_BOVIN^Q:586-263,H:2-107^73.1...",.,SOAP.k61.C38569.p2,119-577[-],"S10A9_BOVIN^S10A9_BOVIN^Q:1-105,H:5-107^73.3%I...",tr|L5MD39|L5MD39_MYODS^tr|L5MD3

## DECockroach

In [10]:
anno_DECockroach = pd.read_csv('truth/DECockroach_ann_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_DECockroach['all_GO'] = anno_DECockroach['gene_ontology_BLASTX'] + '`' + anno_DECockroach['gene_ontology_BLASTP'] + "`" +  anno_DECockroach['gene_ontology_Pfam']
DECockroach_truth_set = anno_DECockroach[ anno_DECockroach.all_GO.str.contains ('|'.join(amp_go_df.ID.to_list()))]
DECockroach_truth_set = DECockroach_truth_set[DECockroach_truth_set["prot_id"] != "."]
DECockroach_truth_set.to_csv("truth/DECockroach_truth_set.csv", columns=BAT_truth_set.columns ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)
DECockroach_truth_set

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,uniprot_metazoa_33208.fasta_BLASTX,uniprot_metazoa_33208.fasta_BLASTP,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_BLASTX,gene_ontology_BLASTP,gene_ontology_Pfam,transcript,peptide,all_GO
87,TRINITY_DN18302_c0_g3_i1,TRINITY_DN18302_c0_g3_i1,"LAMB1_DROME^LAMB1_DROME^Q:147-5474,H:5-1786^50...",.,TRINITY_DN18302_c0_g3_i1.p1,123-5483[+],"LAMB1_DROME^LAMB1_DROME^Q:9-1784,H:5-1786^50.5...",tr|A0A1V1FV97|A0A1V1FV97_9NEOP^tr|A0A1V1FV97|A...,tr|A0A1V1FV97|A0A1V1FV97_9NEOP^tr|A0A1V1FV97|A...,PF00055.18^Laminin_N^Laminin N-terminal (Domai...,.,.,.,KEGG:dme:Dmel_CG7123,GO:0005604^cellular_component^basement membran...,GO:0005604^cellular_component^basement membran...,.,.,.,GO:0005604^cellular_component^basement membran...
317,TRINITY_DN21693_c0_g1_i1,TRINITY_DN21693_c0_g1_i1,"FAT2_DROME^FAT2_DROME^Q:312-19,H:3528-3625^66....",.,TRINITY_DN21693_c0_g1_i1.p1,1-312[-],"FAT2_DROME^FAT2_DROME^Q:1-98,H:3528-3625^66.3%...",tr|A0A2J7Q2R9|A0A2J7Q2R9_9NEOP^tr|A0A2J7Q2R9|A...,tr|A0A2J7Q2R9|A0A2J7Q2R9_9NEOP^tr|A0A2J7Q2R9|A...,.,.,.,.,KEGG:dme:Dmel_CG7749,GO:0098858^cellular_component^actin-based cell...,GO:0098858^cellular_component^actin-based cell...,.,.,.,GO:0098858^cellular_component^actin-based cell...
521,TRINITY_DN24079_c0_g1_i1,TRINITY_DN24079_c0_g1_i1,"NU301_DROME^NU301_DROME^Q:12-353,H:615-728^56....",.,TRINITY_DN24079_c0_g1_i1.p1,3-353[+],"NU301_DROME^NU301_DROME^Q:4-117,H:615-728^56.1...",tr|A0A2J7RD28|A0A2J7RD28_9NEOP^tr|A0A2J7RD28|A...,tr|A0A2J7RD28|A0A2J7RD28_9NEOP^tr|A0A2J7RD28|A...,.,.,.,.,KEGG:dme:Dmel_CG32346,GO:0005634^cellular_component^nucleus`GO:00165...,GO:0005634^cellular_component^nucleus`GO:00165...,.,.,.,GO:0005634^cellular_component^nucleus`GO:00165...
539,TRINITY_DN5073_c0_g2_i1,TRINITY_DN5073_c0_g2_i1,"SPON2_RAT^SPON2_RAT^Q:73-750,H:16-245^43.9%ID^...",.,TRINITY_DN5073_c0_g2_i1.p1,25-1359[+],"SPON2_RAT^SPON2_RAT^Q:17-242,H:16-245^43.9%ID^...",tr|A0A2J7Q4H2|A0A2J7Q4H2_9NEOP^tr|A0A2J7Q4H2|A...,tr|A0A2J7Q4H2|A0A2J7Q4H2_9NEOP^tr|A0A2J7Q4H2|A...,PF06468.14^Spond_N^Spondin_N^42-231^E:8.8e-73`...,.,.,.,KEGG:rno:100910790`KEGG:rno:171569,GO:0031012^cellular_component^extracellular ma...,GO:0031012^cellular_component^extracellular ma...,.,.,.,GO:0031012^cellular_component^extracellular ma...
617,TRINITY_DN6676_c0_g4_i1,TRINITY_DN6676_c0_g4_i1,"TOLL7_DROME^TOLL7_DROME^Q:5-2302,H:633-1366^46...",.,TRINITY_DN6676_c0_g4_i1.p1,305-2416[+],"TOLL7_DROME^TOLL7_DROME^Q:5-666,H:736-1366^43....",tr|A0A2P8XAF7|A0A2P8XAF7_BLAGE^tr|A0A2P8XAF7|A...,tr|A0A2P8XAF7|A0A2P8XAF7_BLAGE^tr|A0A2P8XAF7|A...,PF13855.7^LRR_8^Leucine rich repeat^101-158^E:...,.,.,.,KEGG:dme:Dmel_CG8595,GO:0009897^cellular_component^external side of...,GO:0009897^cellular_component^external side of...,GO:0005515^molecular_function^protein binding`...,.,.,GO:0009897^cellular_component^external side of...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53996,SOAP.k33.C294040,SOAP.k33.C294040,"DORS_DROME^DORS_DROME^Q:323-1180,H:44-329^76.2...",.,SOAP.k33.C294040.p1,206-2131[+],"DORS_DROME^DORS_DROME^Q:40-325,H:44-329^76.2%I...",tr|A0A2P8XXF4|A0A2P8XXF4_BLAGE^tr|A0A2P8XXF4|A...,tr|A0A2P8XXF4|A0A2P8XXF4_BLAGE^tr|A0A2P8XXF4|A...,PF00554.23^RHD_DNA_bind^Rel homology DNA-bindi...,.,.,.,KEGG:dme:Dmel_CG6667,GO:0005737^cellular_component^cytoplasm`GO:000...,GO:0005737^cellular_component^cytoplasm`GO:000...,GO:0003677^molecular_function^DNA binding`GO:0...,.,.,GO:0005737^cellular_component^cytoplasm`GO:000...
54000,SOAP.k33.C294190,SOAP.k33.C294190,"G3BP2_MOUSE^G3BP2_MOUSE^Q:189-2099,H:1-476^33%...",.,SOAP.k33.C294190.p1,189-2135[+],"G3BP2_MOUSE^G3BP2_MOUSE^Q:1-637,H:1-476^33%ID^...",tr|A0A2P8XIY8|A0A2P8XIY8_BLAGE^tr|A0A2P8XIY8|A...,tr|A0A2P8XIY8|A0A2P8XIY8_BLAGE^tr|A0A2P8XIY8|A...,PF02136.21^NTF2^Nuclear transport factor 2 (NT...,.,.,.,KEGG:mmu:23881,GO:0010494^cellular_component^cytoplasmic stre...,GO:0010494^cellular_component^cytoplasmic stre...,GO:00036

In [11]:
# amp_go_stringent_df
anno_DECockroach = pd.read_csv('truth/DECockroach_ann_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_DECockroach['all_GO'] = anno_DECockroach['gene_ontology_BLASTX'] + '`' + anno_DECockroach['gene_ontology_BLASTP'] + "`" +  anno_DECockroach['gene_ontology_Pfam']
DECockroach_truth_set = anno_DECockroach[ anno_DECockroach.all_GO.str.contains ('|'.join(amp_go_stringent_df.ID.to_list()))]
DECockroach_truth_set = DECockroach_truth_set[DECockroach_truth_set["prot_id"] != "."]
DECockroach_truth_set.to_csv("truth/DECockroach_truth_set.stringent.csv", columns=BAT_truth_set.columns ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)
DECockroach_truth_set

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,uniprot_metazoa_33208.fasta_BLASTX,uniprot_metazoa_33208.fasta_BLASTP,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_BLASTX,gene_ontology_BLASTP,gene_ontology_Pfam,transcript,peptide,all_GO
539,TRINITY_DN5073_c0_g2_i1,TRINITY_DN5073_c0_g2_i1,"SPON2_RAT^SPON2_RAT^Q:73-750,H:16-245^43.9%ID^...",.,TRINITY_DN5073_c0_g2_i1.p1,25-1359[+],"SPON2_RAT^SPON2_RAT^Q:17-242,H:16-245^43.9%ID^...",tr|A0A2J7Q4H2|A0A2J7Q4H2_9NEOP^tr|A0A2J7Q4H2|A...,tr|A0A2J7Q4H2|A0A2J7Q4H2_9NEOP^tr|A0A2J7Q4H2|A...,PF06468.14^Spond_N^Spondin_N^42-231^E:8.8e-73`...,.,.,.,KEGG:rno:100910790`KEGG:rno:171569,GO:0031012^cellular_component^extracellular ma...,GO:0031012^cellular_component^extracellular ma...,.,.,.,GO:0031012^cellular_component^extracellular ma...
617,TRINITY_DN6676_c0_g4_i1,TRINITY_DN6676_c0_g4_i1,"TOLL7_DROME^TOLL7_DROME^Q:5-2302,H:633-1366^46...",.,TRINITY_DN6676_c0_g4_i1.p1,305-2416[+],"TOLL7_DROME^TOLL7_DROME^Q:5-666,H:736-1366^43....",tr|A0A2P8XAF7|A0A2P8XAF7_BLAGE^tr|A0A2P8XAF7|A...,tr|A0A2P8XAF7|A0A2P8XAF7_BLAGE^tr|A0A2P8XAF7|A...,PF13855.7^LRR_8^Leucine rich repeat^101-158^E:...,.,.,.,KEGG:dme:Dmel_CG8595,GO:0009897^cellular_component^external side of...,GO:0009897^cellular_component^external side of...,GO:0005515^molecular_function^protein binding`...,.,.,GO:0009897^cellular_component^external side of...
642,TRINITY_DN5657_c0_g1_i1,TRINITY_DN5657_c0_g1_i1,"DEFI_TENMO^DEFI_TENMO^Q:84-290,H:3-84^48.2%ID^...",.,TRINITY_DN5657_c0_g1_i1.p1,169-474[+],.,tr|A0A2P8XXT3|A0A2P8XXT3_BLAGE^tr|A0A2P8XXT3|A...,tr|A0A2P8XXT4|A0A2P8XXT4_BLAGE^tr|A0A2P8XXT4|A...,.,.,.,.,.,GO:0005576^cellular_component^extracellular re...,.,.,.,.,GO:0005576^cellular_component^extracellular re...
1111,TRINITY_DN31308_c0_g1_i1,TRINITY_DN31308_c0_g1_i1,"ILRUN_DANRE^ILRUN_DANRE^Q:178-870,H:4-248^51.4...",.,TRINITY_DN31308_c0_g1_i1.p1,178-891[+],"ILRUN_DANRE^ILRUN_DANRE^Q:1-231,H:4-248^51.4%I...",tr|A0A067RRT6|A0A067RRT6_ZOONE^tr|A0A067RRT6|A...,tr|A0A067RRT6|A0A067RRT6_ZOONE^tr|A0A067RRT6|A...,PF14555.7^UBA_4^UBA-like domain^24-64^E:5.2e-1...,.,.,.,KEGG:dre:541415,GO:0005634^cellular_component^nucleus`GO:00004...,GO:0005634^cellular_component^nucleus`GO:00004...,.,.,.,GO:0005634^cellular_component^nucleus`GO:00004...
1416,TRINITY_DN952_c0_g1_i2,TRINITY_DN952_c0_g1_i2,"EXOC2_DROME^EXOC2_DROME^Q:4120-1418,H:1-889^48...",.,TRINITY_DN952_c0_g1_i2.p1,1373-4120[-],"EXOC2_DROME^EXOC2_DROME^Q:1-901,H:1-889^48.4%I...",tr|A0A067QEV0|A0A067QEV0_ZOONE^tr|A0A067QEV0|A...,tr|A0A067QEV0|A0A067QEV0_ZOONE^tr|A0A067QEV0|A...,PF01833.25^TIG^IPT/TIG domain^5-86^E:8.2e-06`P...,.,.,.,KEGG:dme:Dmel_CG8843,GO:0005642^cellular_component^annulate lamella...,GO:0005642^cellular_component^annulate lamella...,.,.,.,GO:0005642^cellular_component^annulate lamella...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52870,SOAP.k33.scaffold233,SOAP.k33.scaffold233,"PGPLF_DROME^PGPLF_DROME^Q:365-862,H:59-234^34....",.,SOAP.k33.scaffold233.p1,41-874[+],"PGPLF_DROME^PGPLF_DROME^Q:109-274,H:59-234^34....",tr|A0A2P8YL18|A0A2P8YL18_BLAGE^tr|A0A2P8YL18|A...,tr|A0A2P8YL18|A0A2P8YL18_BLAGE^tr|A0A2P8YL18|A...,PF01510.26^Amidase_2^N-acetylmuramoyl-L-alanin...,.,.,.,KEGG:dme:Dmel_CG4437,GO:0005887^cellular_component^integral compone...,GO:0005887^cellular_component^integral compone...,GO:0008745^molecular_function^N-acetylmuramoyl...,.,.,GO:0005887^cellular_component^integral compone...
53995,SOAP.k33.C293944,SOAP.k33.C293944,"XRCC5_MOUSE^XRCC5_MOUSE^Q:109-2223,H:7-732^32....",.,SOAP.k33.C293944.p1,94-2226[+],"XRCC5_MOUSE^XRCC5_MOUSE^Q:6-710,H:7-732^32.1%I...",tr|A0A067QUA1|A0A067QUA1_ZOONE^tr|A0A067QUA1|A...,tr|A0A067QUA1|A0A067QUA1_ZOONE^tr|A0A067QUA1|A...,PF03731.16^Ku_N^Ku70/Ku80 N-terminal alpha/bet...,.,.,.,KEGG:mmu:22596,GO:0005737^cellular_component^cytoplasm`GO:004...,GO:0005737^cellular_component^cytoplasm`GO:004...,GO:0003677^molecular_function^DNA binding`GO:0...,.,.,GO:0005737^cellular_c

In [233]:
anno_DECockroach[anno_DECockroach["prot_id"] == "." ]

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,uniprot_metazoa_33208.fasta_BLASTX,uniprot_metazoa_33208.fasta_BLASTP,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_BLASTX,gene_ontology_BLASTP,gene_ontology_Pfam,transcript,peptide
1,TRINITY_DN18621_c2_g2_i1,TRINITY_DN18621_c2_g2_i1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
2,TRINITY_DN18657_c0_g1_i1,TRINITY_DN18657_c0_g1_i1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
3,TRINITY_DN18694_c0_g1_i1,TRINITY_DN18694_c0_g1_i1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
4,TRINITY_DN18694_c0_g2_i1,TRINITY_DN18694_c0_g2_i1,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
5,TRINITY_DN18698_c0_g2_i4,TRINITY_DN18698_c0_g2_i4,.,.,.,.,.,tr|A0A2P8XND9|A0A2P8XND9_BLAGE^tr|A0A2P8XND9|A...,.,.,.,.,.,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54960,SOAP.k37.C206979,SOAP.k37.C206979,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
54967,SOAP.k37.C207631,SOAP.k37.C207631,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
54976,SOAP.k37.C208251,SOAP.k37.C208251,.,.,.,.,.,tr|A0A2P8ZF92|A0A2P8ZF92_BLAGE^tr|A0A2P8ZF92|A...,.,.,.,.,.,.,.,.,.,.,.
54981,SOAP.k37.C208577,SOAP.k37.C208577,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.


## Search Term 

In [187]:
string_tmp =  BAT_truth_set[BAT_truth_set["prot_id"] == "SOAP.k25.C309190.p1"]["gene_ontology_BLASTX"].values[0]

In [188]:
string_tmp

'GO:0070419^cellular_component^nonhomologous end joining complex`GO:0005730^cellular_component^nucleolus`GO:0005634^cellular_component^nucleus`GO:0032040^cellular_component^small-subunit processome`GO:0005524^molecular_function^ATP binding`GO:0003677^molecular_function^DNA binding`GO:0004677^molecular_function^DNA-dependent protein kinase activity`GO:0106310^molecular_function^protein serine kinase activity`GO:0004674^molecular_function^protein serine/threonine kinase activity`GO:0106311^molecular_function^protein threonine kinase activity`GO:0034511^molecular_function^U3 snoRNA binding`GO:0006974^biological_process^cellular response to DNA damage stimulus`GO:0006302^biological_process^double-strand break repair`GO:0006303^biological_process^double-strand break repair via nonhomologous end joining`GO:0033152^biological_process^immunoglobulin V(D)J recombination`GO:0045087^biological_process^innate immune response`GO:0008630^biological_process^intrinsic apoptotic signaling pathway in re

In [195]:
for go_term in amp_go_df.ID.to_list():
    position = string_tmp.find(go_term)
    if(position != -1):
        print(go_term)


GO:0045087


In [199]:
amp_go_df[amp_go_df["ID"] == "GO:0045087"]["definition"].values

array(['Innate immune responses are defense responses mediated by germline encoded components that directly recognize components of potential pathogens.'],
      dtype=object)

# Check seq 
Filter annotation ID that contain ambigious codiing character  

## DECockroach

In [25]:
anno_DECockroach = pd.read_csv('truth/DECockroach_ann_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_DECockroach['all_GO'] = anno_DECockroach['gene_ontology_BLASTX'] + '`' + anno_DECockroach['gene_ontology_BLASTP'] + "`" +  anno_DECockroach['gene_ontology_Pfam']
DECockroach_truth_set = anno_DECockroach[ anno_DECockroach.all_GO.str.contains ('|'.join(amp_go_df.ID.to_list()))]
DECockroach_truth_set = DECockroach_truth_set[DECockroach_truth_set["prot_id"] != "."]

_fasta = "pep/DECockroach_SRR1552488.combined.okay.fa.transdecoder.pep"
with open(_fasta) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    sequence = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequence.append(str(seq_record.seq))
        lengths.append(len(seq_record.seq))

de_df = pd.DataFrame(list(zip(identifiers, sequence, lengths)), columns =['ID', 'Sequence', 'length']) 
_de_amp_df= de_df[de_df['ID'].isin(DECockroach_truth_set.prot_id.to_list())]
_de_amp_df

NON_CODE = "B|Z|J|U|O|X"
_de_amp_df = _de_amp_df[~_de_amp_df["Sequence"].str.contains(NON_CODE, regex=True)]

final_df = pd.DataFrame(_de_amp_df["ID"].values, columns=['prot_id'])
final_df.to_csv("truth_no_ambiguous/DECockroach_truth_set.csv", columns=["prot_id"] ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)


In [27]:
# stringent
anno_DECockroach = pd.read_csv('truth/DECockroach_ann_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_DECockroach['all_GO'] = anno_DECockroach['gene_ontology_BLASTX'] + '`' + anno_DECockroach['gene_ontology_BLASTP'] + "`" +  anno_DECockroach['gene_ontology_Pfam']
DECockroach_truth_set = anno_DECockroach[ anno_DECockroach.all_GO.str.contains ('|'.join(amp_go_stringent_df.ID.to_list()))]
DECockroach_truth_set = DECockroach_truth_set[DECockroach_truth_set["prot_id"] != "."]

_fasta = "pep/DECockroach_SRR1552488.combined.okay.fa.transdecoder.pep"
with open(_fasta) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    sequence = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequence.append(str(seq_record.seq))
        lengths.append(len(seq_record.seq))

de_df = pd.DataFrame(list(zip(identifiers, sequence, lengths)), columns =['ID', 'Sequence', 'length']) 
_de_amp_df= de_df[de_df['ID'].isin(DECockroach_truth_set.prot_id.to_list())]
_de_amp_df

NON_CODE = "B|Z|J|U|O|X"
_de_amp_df = _de_amp_df[~_de_amp_df["Sequence"].str.contains(NON_CODE, regex=True)]

final_df = pd.DataFrame(_de_amp_df["ID"].values, columns=['prot_id'])
final_df.to_csv("truth_no_ambiguous/DECockroach_truth_set.stringent.csv", columns=["prot_id"] ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)

## BAT

In [29]:
anno_BAT = pd.read_csv('truth/BAT_annotation_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_BAT['all_GO'] = anno_BAT['gene_ontology_BLASTX'] + '`' + anno_BAT['gene_ontology_BLASTP'] + "`" +  anno_BAT['gene_ontology_Pfam']
BAT_truth_set = anno_BAT[ anno_BAT.all_GO.str.contains('|'.join(amp_go_df.ID.to_list()))]
BAT_truth_set = BAT_truth_set[BAT_truth_set["prot_id"] != "."]


_fasta = "pep/BAT_SRR12103592.combined.okay.fa.transdecoder.pep"
with open(_fasta) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    sequence = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequence.append(str(seq_record.seq))
        lengths.append(len(seq_record.seq))

bat_df = pd.DataFrame(list(zip(identifiers, sequence, lengths)), columns =['ID', 'Sequence', 'length']) 
_bat_amp_df= bat_df[bat_df['ID'].isin(BAT_truth_set.prot_id.to_list())]
NON_CODE = "B|Z|J|U|O|X"
_bat_amp_df = _bat_amp_df[~_bat_amp_df["Sequence"].str.contains(NON_CODE, regex=True)]
final_df = pd.DataFrame(_bat_amp_df["ID"].values, columns=['prot_id'])
final_df.to_csv("truth_no_ambiguous/BAT_truth_set.csv", columns=["prot_id"] ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)

In [31]:
anno_BAT = pd.read_csv('truth/BAT_annotation_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_BAT['all_GO'] = anno_BAT['gene_ontology_BLASTX'] + '`' + anno_BAT['gene_ontology_BLASTP'] + "`" +  anno_BAT['gene_ontology_Pfam']
BAT_truth_set = anno_BAT[ anno_BAT.all_GO.str.contains('|'.join(amp_go_stringent_df.ID.to_list()))]
BAT_truth_set = BAT_truth_set[BAT_truth_set["prot_id"] != "."]


_fasta = "pep/BAT_SRR12103592.combined.okay.fa.transdecoder.pep"
with open(_fasta) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    sequence = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequence.append(str(seq_record.seq))
        lengths.append(len(seq_record.seq))

bat_df = pd.DataFrame(list(zip(identifiers, sequence, lengths)), columns =['ID', 'Sequence', 'length']) 
_bat_amp_df= bat_df[bat_df['ID'].isin(BAT_truth_set.prot_id.to_list())]
NON_CODE = "B|Z|J|U|O|X"
_bat_amp_df = _bat_amp_df[~_bat_amp_df["Sequence"].str.contains(NON_CODE, regex=True)]
final_df = pd.DataFrame(_bat_amp_df["ID"].values, columns=['prot_id'])
final_df.to_csv("truth_no_ambiguous/BAT_truth_set.stringent.csv", columns=["prot_id"] ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)